![ROOT Logo](http://root.cern.ch/img/logos/ROOT_Logo/website-banner/website-banner-%28not%20root%20picture%29.jpg)
![Spark Logo](http://spark-mooc.github.io/web-assets/images/ta_Spark-logo-small.png)
# **DistROOT: Example Notebook**
<hr style="border-top-width: 4px; border-top-color: #34609b;">

Import the necessary packages.

In [ ]:
import ROOT
from DistROOT import DistTree

Define the mapper and reducer functions.

In [ ]:
%%cpp -d

TH2F* fillHist(TTreeReader& reader) {
    auto px = TTreeReaderValue<float>(reader, "px");
    auto py = TTreeReaderValue<float>(reader, "py");
    
    TH2F *h = new TH2F("hpxpy", "py vs px", 40, -4, 4, 40, -4, 4);
    while (reader.Next()) {
        h->Fill(*px, *py);
    }
    
    return h;
}

TH2F* mergeHist(TH2F *h1, TH2F *h2) {
    h1->Add(h2);
    return h1;
}

Build the DistTree and trigger the parallel processing.

In [ ]:
dTree = DistTree(filelist = ["http://root.cern.ch/files/test_distroot_small.root"],
                 treename = "test",
                 npartitions = 2)

myHist = dTree.ProcessAndMerge(ROOT.fillHist, ROOT.mergeHist)

Plot the resulting histogram.

In [ ]:
c = ROOT.TCanvas("myCanvas", "Test Histogram", 800, 600)
myHist.Draw()
c.Draw()